In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
directory = '../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset/'

In [3]:
#Labels
filepath = os.listdir(directory)
print(filepath)

In [4]:
classes = ['Hourse Mackerel', 'Black Sea Sprat', 'Sea Bass', 'Red Mullet', 'Trout',
             'Striped Red Mullet', 'Shrimp', 'Gilt-Head Bream', 'Red Sea Bream']

In [5]:
label = []
path = []
for dir_name, _, filenames in os.walk(directory):
    for filename in filenames:
        if os.path.splitext(filename)[1]=='.png':
            if dir_name.split()[-1] != 'GT':
                label.append(os.path.split(dir_name)[1])
                path.append(os.path.join(dir_name, filename))
data = pd.DataFrame(columns=['path','label'])
data['path'] = path
data['label'] = label

In [6]:
data.head()

In [7]:
import seaborn as sns

In [8]:
data['label'] = data['label'].astype('category')

In [9]:
import matplotlib.pyplot as plt

In [10]:
data['label'].value_counts()

In [11]:
plt.figure(figsize=(20,8))
sns.countplot(data=data, x='label')

In [12]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(15,8), constrained_layout=True)
ax = ax.flatten()
idx=0
for i in data['label'].unique():
    ax[idx].imshow(plt.imread(data[data['label']==i].iloc[0,0]))
    ax[idx].set_title(i)
    idx+=1

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train, test = train_test_split(data, test_size=0.2)

In [15]:
print(train.shape)
print(test.shape)

In [16]:
import torch
import torch.nn as nn
import torchvision
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from torchvision.models import vgg16

**Model**

In [17]:
class ImageTransform():
    def __init__(self, input_size):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(input_size),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'test': transforms.Compose([
                    transforms.Resize(input_size),
                    transforms.CenterCrop(input_size),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        }
        
    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)

In [18]:
input_size = 224

In [19]:
img_file_path = '../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset/Gilt-Head Bream/Gilt-Head Bream/00001.png'
img = Image.open(img_file_path)
plt.axis('off')
plt.imshow(img)
plt.show()

transform = ImageTransform(input_size)
img_transformed = transform(img, phase='train')

# (channel, height, width) -> (height, width, channel)
img_transformed = img_transformed.permute(1, 2, 0)
plt.imshow(img_transformed)
plt.axis('off')
plt.show()

In [20]:
idx_to_class = {i:j for i, j in enumerate(classes)}
class_to_idx = {value:key for key,value in idx_to_class.items()}

In [21]:
print(idx_to_class.items())

In [22]:
print(class_to_idx)

In [23]:
# test
a = '../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat GT'
print(a.split()[-1])
if a.split()[-1] != 'GT':
    label = os.path.split(a)[1]
    print(label)

In [24]:
train.iloc[1,0]

In [25]:
class MyDataset(Dataset):
    def __init__(self, img_path, transform=None, phase='train'):
        self.img_path = img_path
        self.transform = transform
        self.phase = phase
        
    def __len__(self):
        return len(self.img_path)
    
    def __getitem__(self, idx):
        img_file_path = self.img_path.iloc[idx,0]
        img = Image.open(img_file_path)
        
        img_transformed = self.transform(img, self.phase)
        
        label = self.img_path.iloc[idx,1]
        label = class_to_idx[label]
            
        return img_transformed, label

In [26]:
train_ds = MyDataset(train, transform=ImageTransform(input_size), phase='train')
test_ds = MyDataset(test, transform=ImageTransform(input_size), phase='test')

In [27]:
index = 0
img, label = train_ds.__getitem__(index)
print(img.size())
print(label)

In [28]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [29]:
batch_size = 32

train_dataloader = DataLoader(train_ds, batch_size,shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size, shuffle=False)

dataloader_dict = {"train": train_dataloader, 'test': test_dataloader}

In [30]:
batch_iteration = iter(dataloader_dict['train'])
inputs, labels = next(batch_iteration)

In [31]:
print(inputs.size())
print(labels)

In [32]:
for img, label in train_dataloader:
    fig, ax = plt.subplots(figsize=(10,8))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(make_grid(img, 6).permute(1,2,0))
    break
    

In [33]:
use_pretrained = True
model = vgg16(pretrained=use_pretrained)
print(model)

In [34]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=9)
print(model)
model = model.to(device)

In [35]:
def params_to_update(model):
    params_to_update_1 = []
    params_to_update_2 = []
    params_to_update_3 = []

    update_param_name_1 = ["features"]
    update_param_name_2 = ["classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
    update_param_name_3 = ["classifier.6.weight", "classifier.6.bias"]

    for name, param in model.named_parameters():
        if name in update_param_name_1:
            param.requires_grad = True
            params_to_update_1.append(param)
        elif name in update_param_name_2:
            param.requires_grad = True
            params_to_update_2.append(param)
        elif name in update_param_name_3:
            param.requires_grad = True
            params_to_update_3.append(param)
        
        else:
            param.requires_grad = False
    return params_to_update_1, params_to_update_2, params_to_update_3

In [36]:
def load_model(model, model_path):
    load_weights = torch.load(model_path,  map_location={"cuda:0": "cpu"})
    model.load_state_dict(load_weights)

    
    return model

In [37]:
from tqdm import tqdm

In [38]:
def train_model(model, dataloader_dict, criterion, optimizer, device):
    model = model.train()
    
    epoch_loss = 0.0
    correct_prediction = 0
    global epoch_accuracy
    for images, labels in tqdm(dataloader_dict):
        
        images = images.to(device)
        labels = labels.to(device)
        
             
        # Forward
        outputs = model(images)
        # Calculate loss
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        
        epoch_loss += loss.item()*images.size(0)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        correct_prediction += torch.sum(preds == labels.data)
        
        epoch_loss = epoch_loss / len(dataloader_dict.dataset)
        epoch_accuracy = correct_prediction.double() / len(dataloader_dict.dataset)
        
    return epoch_accuracy, epoch_loss

In [39]:
def evaluate_epoch(model, dataloader_dict, criterion, optimizer, device):
    model.eval()
    
    epoch_loss = 0.0
    correct_prediction = 0
    global epoch_accuracy
    with torch.no_grad():
        for images, labels in tqdm(dataloader_dict):
            # Load images, labels to device
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            
            _, preds = torch.max(outputs, 1)
            
            epoch_loss += loss.item()*images.size(0)
            correct_prediction += torch.sum(preds == labels.data)
            
            epoch_loss = epoch_loss / len(dataloader_dict.dataset)
            epoch_accuracy = correct_prediction.double() / len(dataloader_dict.dataset)
            
        return epoch_accuracy, epoch_loss

In [40]:
from collections import defaultdict

In [41]:
criterion = nn.CrossEntropyLoss()
# optimizer
params1, params2, params3 = params_to_update(model)
optimizer = optim.SGD([
        {'params': params1, 'lr': 1e-4}, 
        {'params': params2, 'lr': 5e-4},
        {'params': params3, 'lr': 1e-3}, 
    ], momentum=0.9)

In [43]:
%%time
history = defaultdict(list)
best_val_acc = 0.0
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
EPOCHS = 3

for epoch in range(EPOCHS):
    print(f'\nEpoch: [{epoch+1}/{EPOCHS}]')
    print('-'*30)
    
    train_acc, train_loss = train_model(model, dataloader_dict['train'], criterion, optimizer,device)
    val_acc, val_loss = evaluate_epoch(model, dataloader_dict['test'], criterion, optimizer, device)
    
    print('Train Loss: {:.4f}\t Train Acc: {:.4f}'.format(train_loss, train_acc))
    print('Val Loss: {:.4f}\t Val Acc: {:.4f}'.format(val_loss, val_acc))
    
    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_restnet.pth')
        

In [45]:
# Plot results
plt.title('Loss')
plt.plot(np.arange(1,4), history['train_loss'], color='blue', label='Train loss')
plt.plot(np.arange(1,4), history['val_loss'], color='red', label='Test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.title('Accuracy')
plt.plot(np.arange(1,4), history['train_acc'], 'bo--', label='Train Acc')
plt.plot(np.arange(1,4), history['val_acc'], color='red', label='Test Acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [54]:
class Predictor():
    def __init__(self, classes):
        self.class_index = classes
        
    def predict_max(self, output):
        max_id = np.argmax(output.detach().numpy())
        predicted_label = self.class_index[max_id]
        return predicted_label
    
predictor = Predictor(classes)

def predict(img):
    use_pretrained = True
    model = vgg16(pretrained=use_pretrained)
    model.classifier[6] = nn.Linear(in_features=4096, out_features=9, bias=True)
    model.eval()
    
    
    model = load_model(model, './best_restnet.pth')
    
    #prepare inputdata
    '''
    img = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    '''
    transform = ImageTransform(input_size)
    img = transform(img, phase="test")
    img = img.unsqueeze(0)
    
    output = model(img)
    response = predictor.predict_max(output)
    
    return response

In [57]:
test_img = Image.open('../input/a-large-scale-fish-dataset/NA_Fish_Dataset/Horse Mackerel/00003.png').convert('RGB')
plt.imshow(test_img)
plt.show()
print('Predicted image: ' + predict(test_img))